In [3]:
!pip install keras
!pip install tensorflow
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.models import Model
from keras.layers import Input, Embedding, SimpleRNN, Dense, LSTM, GRU, Bidirectional

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached google_auth-2.28.1-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached Markdown-3.5.2-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.0.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl.metadata (3.6 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl.metadata (10 kB)
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl.metadata (8.6 kB)
  Using ca

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [4]:
df = pd.read_json('final_dataset.json')

In [5]:
df.head()

,overall,vote,verified,reviewerID,asin,style,reviewerName,reviewText,summary,has_image,clean_reviewText,clean_summary,reviewText_length,clean_reviewText_length,review_emotions,review_emotion_scores,dominant_emotion,inferred_intent
0,1,0,True,A2KGOFRA94OWS9,B00GZOI1OC,Unknown,jlpoole520,"The part (the biggest part, the wheel) didn't ...","The part (the biggest part, the wheel) didn't ...",0,part biggest part wheel didnt fit buy new mach...,part biggest part wheel didnt fit,49,22,"[realization, fear, confusion, pride, neutral]","[0.336, 0.225, 0.221, 0.044, 0.043]",realization,Inquiry/Support Intent
1,1,0,True,A1LIJY0J529J27,B00LHQYW0A,Unknown,John P,Incomplete order. Picture shows two heating e...,"Incomplete order, missing one heating element",0,incomplete order picture show two heating elem...,incomplete order missing one heating element,11,9,"[pride, embarrassment, anger, neutral, nervous...","[0.817, 0.067, 0.049, 0.029, 0.010]",pride,Praise/Endorsement Intent
2,1,0,False,A3OMTDM4HR5XV9,B0050KMOM2,Unknown,steve combs,best bushing I've had in a long time,best bushing I've had in a long time,0,best bushing ive long time,best bushing ive long time,8,5,"[disappointment, disapproval, pride, confusion...","[0.990, 0.004, 0.001, 0.001, 0.000]",disappointment,Problem/Issue Reporting Intent
3,1,0,False,A10M3NOJHRC4F4,B001GD43RG,Unknown,Amazon Customer,"Biggest piece of s*** we ever bought, replaced...","Biggest piece of s*** we ever bought, replaced...",0,biggest piece ever bought replaced motor 4 tim...,biggest piece ever bought replaced motor,22,14,"[pride, disapproval, disappointment, excitemen...","[0.994, 0.001, 0.001, 0.001, 0.000]",pride,Praise/Endorsement Intent
4,1,0,True,A3LF8E0TB9XWMZ,B00LJ5K82K,Unknown,Amazon Customer,dud shorted out element burned out the same da...,not very happy,0,dud shorted element burned daynot happy,happy,11,6,"[surprise, anger, pride, neutral, embarrassment]","[0.520, 0.283, 0.077, 0.038, 0.026]",surprise,Comparison/Consideration Intent


In [6]:
# Display the first few results
print(df[['reviewText', 'clean_reviewText']].head())

                                          reviewText  \
0  The part (the biggest part, the wheel) didn't ...   
1  Incomplete order.  Picture shows two heating e...   
2             best  bushing  I've had in a long time   
3  Biggest piece of s*** we ever bought, replaced...   
4  dud shorted out element burned out the same da...   

                                    clean_reviewText  
0  part biggest part wheel didnt fit buy new mach...  
1  incomplete order picture show two heating elem...  
2                         best bushing ive long time  
3  biggest piece ever bought replaced motor 4 tim...  
4            dud shorted element burned daynot happy  


Label Preparation

Overall sentiment Score

In [7]:
from sklearn.preprocessing import MinMaxScaler

#Since the overall values in the overall column on the dataset are between a scale of 1 and 5 we have to normalize to 0-100
scaler_overall = MinMaxScaler(feature_range=(0, 100))
y_overall = scaler_overall.fit_transform(df[['overall']])

Helpfulness of the review

In [8]:
y_vote = df['vote'].fillna(0).values  # Assuming NaN means no votes

# Or convert to binary helpful/unhelpful based on a threshold, e.g., at least one vote to be considered helpful
y_helpfulness = (df['vote'].fillna(0) > 0).astype(int).values

Intent of the review

In [10]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

#Converting the cetegorical inferred_intent to one-hot encoded vectors 
label_encoder_intent = LabelEncoder()
y_intent = label_encoder_intent.fit_transform(df['inferred_intent'])
y_intent = to_categorical(y_intent)

Emotions and their scores 

For emotions, since there are 5 different emotions along with their scores, ensure they are properly formatted

In [11]:
import numpy as np

num_emotions = 5  # Assuming there are always 5 emotions listed
max_emotions = len(set(df['review_emotions'].explode()))  # Total unique emotions across dataset

# Convert emotion names to indices and scores to float
emotion_to_index = {emotion: idx for idx, emotion in enumerate(sorted(set(df['review_emotions'].explode())))}

#Initialize the empty arrays for one-hot encoded emotions and their scores
y_emotions = np.zeros((len(df), num_emotions), dtype=int)
y_emotion_scores = np.zeros((len(df), num_emotions), dtype=float)

# Populate the arrays
for i, (emotions, scores) in enumerate(zip(df['review_emotions'], df['review_emotion_scores'])):
    for j, emotion in enumerate(emotions):
        if j < num_emotions:  # Safety check
            y_emotions[i, j] = emotion_to_index[emotion]
            y_emotion_scores[i, j] = scores[j]

Aspect detection using the clean_summary column